# Deep Learning - A Simple GAN Example in PyTorch
---

## Author : Amir Atapour-Abarghouei, amir.atapour-abarghouei@durham.ac.uk

This notebook will provide an example that shows the implementation of a simple Generative Adversarial Network (GAN) in PyTorch.

Copyright (c) 2023 Amir Atapour-Abarghouei, UK.

License : LGPL - http://www.gnu.org/licenses/lgpl.html

We are going to implement a simple GAN. Let's start by importing what we need:

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

We should now line up the dataset we are going to use. We will be working with our own dataset:

**[AckBinks: A Star Wars Dataset](https://github.com/atapour/dl-pytorch/tree/main/2.Datasets/AckBinks)**

First, let's download the dataset, which is publicly available on GitHub: 

In [ ]:
!wget -q -O AckBinks.zip https://github.com/atapour/dl-pytorch/blob/main/2.Datasets/AckBinks/AckBinks.zip?raw=true
!unzip -q AckBinks.zip
!rm AckBinks.zip
print('done!')

Now we are ready to process our data. We are going to convert our data to 32x32 grayscale images to make the work easier and more efficient just for demonstration purposes.

Since our dataset has two classes, we are also going to pick one of these classes and have our GAN generate images of that class only. Let's pick Jar Jar Binks since he is better-looking:

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize([32,32]),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
])

dataset = torchvision.datasets.ImageFolder('AckBinks/train', transform)

# picking Jar Jar Binks as the class to generate:
idx = torch.tensor(dataset.targets) == 1

train_dataset = torch.utils.data.dataset.Subset(dataset, np.where(idx == 1)[0])

train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=8, shuffle=True, num_workers = 2)

print(f"There are {len(train_dataset)} images in the training set!")

Our dataset is tiny and is not really suited for any real applications, but it will demonstrate the process. Let's look at a few of our images:

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_loader.dataset[i][0].clamp(0,1).permute(0,2,1).contiguous().permute(2,1,0), cmap=plt.cm.binary)

We can now create our models, we need a Generator and a Discriminator:

In [ ]:
# A few parameters:
n_channels = 3
img_width = 32

# define the generator
class Generator(nn.Module):
    def __init__(self, latent_size=100):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(latent_size, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, n_channels*img_width*img_width),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.layer(x)
        x = x.view(x.size(0), n_channels, img_width, img_width)
        return x

# define the discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_channels*img_width*img_width, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1), # 1 output for real/fake
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layer(x)
        return x
        
G = Generator().to(device)
D = Discriminator().to(device)

print(f'Generator has {len(torch.nn.utils.parameters_to_vector(G.parameters()))} parameters.')
print(f'Discriminator has {len(torch.nn.utils.parameters_to_vector(D.parameters()))} parameters')

# initialise the optimiser
optimiser_G = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimiser_D = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
print('Optimisers have been created!')

criterion = nn.BCELoss()
epoch = 0
print('Loss function is Binary Cross Entropy!')


Let's start the main training loop now:

In [ ]:
# training loop
while (epoch<20000):
    
    # arrays for metrics
    logs = {}
    gen_loss_arr = np.zeros(0)
    dis_loss_arr = np.zeros(0)

    # iterate over the train dateset
    for i, batch in enumerate(train_loader):

        x, t = batch
        x, t = x.to(device), t.to(device)

        # train discriminator 
        g = G(torch.randn(x.size(0), 100).to(device))
        l_r = criterion(D(x).mean(), torch.ones(1)[0].to(device)) # real -> 1
        l_f = criterion(D(g.detach()).mean(), torch.zeros(1)[0].to(device)) #  fake -> 0
        loss_d = (l_r + l_f)/2.0
        optimiser_D.zero_grad()
        loss_d.backward()
        optimiser_D.step()
        
        # train generator
        g = G(torch.randn(x.size(0), 100).to(device))
        loss_g = criterion(D(g).mean(), torch.ones(1)[0].to(device)) # fake -> 1
        optimiser_G.zero_grad()
        loss_g.backward()
        optimiser_G.step()

        gen_loss_arr = np.append(gen_loss_arr, loss_g.item())
        dis_loss_arr = np.append(dis_loss_arr, loss_d.item())

    # plot some examples
    G.eval()
    g = G(torch.randn(x.size(0), 100).to(device))
    print('loss D: {:.3f}, loss G: {:.3f}'.format(gen_loss_arr.mean(), dis_loss_arr.mean()))
    plt.grid(False)
    plt.imshow(torchvision.utils.make_grid(g).cpu().data.clamp(0,1).permute(0,2,1).contiguous().permute(2,1,0), cmap=plt.cm.binary)
    plt.show()
    plt.pause(0.0001)
    G.train()

    epoch = epoch+1